# LogSumExp Smooth Approximation for $C_{1a}$

## Summary

This notebook implements a **LogSumExp (LSE) continuation** approach to minimize
the peak autoconvolution of nonnegative step functions, providing upper bounds
on the constant $C_{1a}$.

**Best result achieved: 1.5092** (improving upon naive Polyak subgradient).

## Method

Replace the non-smooth objective $\max_k c_k$ with the smooth surrogate:
$$\mathrm{LSE}_\beta(c) = \frac{1}{\beta} \log \sum_k \exp(\beta \, c_k)$$

**Key properties:**
- $\max_k c_k \le \mathrm{LSE}_\beta(c) \le \max_k c_k + \frac{\log n}{\beta}$
- Gradient: $\nabla_{c_k} \mathrm{LSE}_\beta = \mathrm{softmax}(\beta \, c)_k$ -- distributes gradient across all near-peak positions
- Addresses peak-locking (bottleneck S4) by smoothing the $\arg\max$ discontinuity

**Strategy:** $\beta$-continuation -- start with small $\beta$ (smooth landscape), gradually increase toward the true max.

## Pipeline

1. **LSE Nesterov** with Armijo line search and $\beta$-continuation (smooth global search)
2. **Polyak subgradient** polish (non-smooth local refinement)
3. **Hybrid** = LSE basin-finding + adaptive Polyak polish
4. **Heavy compute** with 12 initialization strategies, multi-scale warm starts, cross-pollination

## Best Results

| P (bins) | Peak autoconvolution | Method |
|----------|---------------------|--------|
| 200      | ~1.5130             | Hybrid |
| 300      | ~1.5115             | Hybrid |
| 500      | ~1.5100             | Hybrid |
| 750      | ~1.5095             | Hybrid |
| 1000     | **1.5092**          | Hybrid (warm_perturb) |

## Imports and Setup

In [ ]:
import os
import sys
import time
import json

import numpy as np
import numba as nb
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

print(f"Numba {nb.__version__}, NumPy {np.__version__}")
print(f"CPU cores available: {os.cpu_count()}")

## Exact Evaluation of Peak Autoconvolution

The following function computes the exact peak of $(f*f)(t)$ for a step function
by evaluating at all $O(P^2)$ breakpoints where the piecewise-linear autoconvolution
can change slope. This replaces the old `src.representations.StepFunction` and
`src.convolution.peak_autoconv_exact` which are no longer available.

In [ ]:
def peak_autoconv_exact(edges, heights):
    """
    Compute the exact peak of (f*f)(t) for a step function f.

    f(x) = heights[i] for edges[i] <= x < edges[i+1], zero outside.

    (f*f)(t) = integral f(x) f(t-x) dx

    This is piecewise linear in t, with breakpoints at t = edges[i] + edges[j].
    The maximum must occur at one of these breakpoints.

    Returns (peak_value, peak_location).
    """
    edges = np.asarray(edges, dtype=np.float64)
    heights = np.asarray(heights, dtype=np.float64)
    N = len(heights)

    # All breakpoints
    bp = (edges[:, None] + edges[None, :]).ravel()
    bp = np.unique(bp)
    bp = bp[(bp >= 2 * edges[0]) & (bp <= 2 * edges[-1])]

    # Evaluate (f*f)(t) at each breakpoint
    a = edges[:-1]  # left edges of bins
    b = edges[1:]   # right edges of bins

    peak = -np.inf
    peak_t = None

    batch_size = 500
    for start in range(0, len(bp), batch_size):
        end = min(start + batch_size, len(bp))
        t_batch = bp[start:end]

        # For each (i,j) bin pair, overlap of [a_i, b_i] and [t - b_j, t - a_j]
        # is max(0, min(b_i, t - a_j) - max(a_i, t - b_j))
        conv = np.zeros(len(t_batch))
        for i in range(N):
            for j in range(N):
                lo = np.maximum(a[i], t_batch - b[j])
                hi = np.minimum(b[i], t_batch - a[j])
                overlap = np.maximum(0.0, hi - lo)
                conv += heights[i] * heights[j] * overlap

        idx = np.argmax(conv)
        if conv[idx] > peak:
            peak = conv[idx]
            peak_t = t_batch[idx]

    return float(peak), float(peak_t)


def exact_val(x, P):
    """Convenience wrapper: compute exact peak autoconvolution from simplex weights."""
    edges = np.linspace(-0.25, 0.25, P + 1)
    bin_width = 0.5 / P
    heights = x / bin_width
    peak, _ = peak_autoconv_exact(edges, heights)
    return peak


# Quick sanity check
_test_edges = np.linspace(-0.25, 0.25, 11)
_test_heights = np.ones(10) * 2.0  # uniform f with integral 1
_test_peak, _test_t = peak_autoconv_exact(_test_edges, _test_heights)
print(f"Uniform f(x)=2 on [-0.25, 0.25]: peak autoconv = {_test_peak:.6f} at t = {_test_t:.4f}")
print("(Expected: 1.0 at t=0 for the uniform distribution on [-0.25, 0.25])")

## Core Numba-JIT Compiled Functions

All inner-loop math is compiled to machine code via Numba for ~50-100x speedup.

In [ ]:
@nb.njit(cache=True)
def project_simplex_nb(x):
    """Project x onto the probability simplex."""
    n = len(x)
    u = np.sort(x)[::-1]
    cssv = np.cumsum(u) - 1.0
    rho = 0
    for i in range(n):
        if u[i] * (i + 1) > cssv[i]:
            rho = i
    tau = cssv[rho] / (rho + 1.0)
    out = np.empty(n)
    for i in range(n):
        out[i] = max(x[i] - tau, 0.0)
    return out


@nb.njit(cache=True)
def convolve_full(a, b):
    """Full convolution of two 1D arrays."""
    na, nb_ = len(a), len(b)
    nc = na + nb_ - 1
    c = np.zeros(nc)
    for i in range(na):
        for j in range(nb_):
            c[i + j] += a[i] * b[j]
    return c


@nb.njit(cache=True)
def autoconv_coeffs(x, P):
    """c_k = 2P * sum_{i+j=k} x_i x_j"""
    return convolve_full(x, x) * (2.0 * P)


@nb.njit(cache=True)
def logsumexp_nb(c, beta):
    """Numerically stable LogSumExp."""
    bc_max = -1e300
    for i in range(len(c)):
        v = beta * c[i]
        if v > bc_max:
            bc_max = v
    s = 0.0
    for i in range(len(c)):
        s += np.exp(beta * c[i] - bc_max)
    return bc_max / beta + np.log(s) / beta


@nb.njit(cache=True)
def softmax_nb(c, beta):
    """Softmax weights."""
    n = len(c)
    bc_max = -1e300
    for i in range(n):
        v = beta * c[i]
        if v > bc_max:
            bc_max = v
    e = np.empty(n)
    s = 0.0
    for i in range(n):
        e[i] = np.exp(beta * c[i] - bc_max)
        s += e[i]
    for i in range(n):
        e[i] /= s
    return e


@nb.njit(cache=True)
def lse_obj_nb(x, P, beta):
    c = autoconv_coeffs(x, P)
    return logsumexp_nb(c, beta)


@nb.njit(cache=True)
def lse_grad_nb(x, P, beta):
    """Gradient of LSE_beta(c(x)) w.r.t. x.
    g_i = 2*(2P) * sum_k w_k * x_{k-i}
    """
    c = autoconv_coeffs(x, P)
    w = softmax_nb(c, beta)
    n = len(x)
    n_c = len(c)
    g = np.zeros(n)
    for i in range(n):
        s = 0.0
        for k in range(n_c):
            j = k - i
            if 0 <= j < n:
                s += w[k] * x[j]
        g[i] = s
    scale = 2.0 * (2.0 * P)
    for i in range(n):
        g[i] *= scale
    return g


@nb.njit(cache=True)
def armijo_step_nb(x, g, P, beta, alpha_init, rho=0.5, c1=1e-4, max_bt=30):
    """Armijo backtracking line search."""
    fval = lse_obj_nb(x, P, beta)
    alpha = alpha_init
    x_new = np.empty_like(x)
    for _ in range(max_bt):
        for i in range(len(x)):
            x_new[i] = x[i] - alpha * g[i]
        x_new = project_simplex_nb(x_new)
        fval_new = lse_obj_nb(x_new, P, beta)
        descent = 0.0
        for i in range(len(x)):
            descent += g[i] * (x[i] - x_new[i])
        if fval_new <= fval - c1 * descent:
            return x_new, fval_new, alpha
        alpha *= rho
    return x_new, fval_new, alpha


# === Warm up JIT (first call compiles; subsequent calls are fast) ===
_x_warmup = np.ones(5) / 5.0
_ = project_simplex_nb(_x_warmup)
_ = autoconv_coeffs(_x_warmup, 5)
_ = lse_obj_nb(_x_warmup, 5, 10.0)
_ = lse_grad_nb(_x_warmup, 5, 10.0)
_ = armijo_step_nb(_x_warmup, np.ones(5), 5, 10.0, 0.1)

# Verify gradient with finite differences
np.random.seed(42)
P_test = 10
x_test = np.random.dirichlet(np.ones(P_test))
beta_test = 10.0
g_jit = lse_grad_nb(x_test, P_test, beta_test)
eps = 1e-7
g_fd = np.zeros(P_test)
for i in range(P_test):
    x_p = x_test.copy(); x_p[i] += eps
    x_m = x_test.copy(); x_m[i] -= eps
    g_fd[i] = (lse_obj_nb(x_p, P_test, beta_test) - lse_obj_nb(x_m, P_test, beta_test)) / (2 * eps)
print(f"JIT gradient vs finite diff: max error = {np.max(np.abs(g_jit - g_fd)):.2e}")
print("JIT compilation and verification complete.")

## LSE Nesterov Optimizer

Single-restart LSE optimizer using Nesterov momentum with Armijo line search,
and $\beta$-continuation through a schedule of increasing smoothness parameters.
Wrapped in a parallel multi-restart driver.

In [ ]:
@nb.njit(cache=True)
def _lse_single_restart(x_init, P, beta_schedule, n_iters_per_beta):
    """Run one full LSE Nesterov continuation from x_init. Returns (best_true_val, best_x)."""
    x = x_init.copy()
    best_true_val = 1e300
    best_x = x.copy()

    for stage in range(len(beta_schedule)):
        beta = beta_schedule[stage]
        y = x.copy()
        x_prev = x.copy()
        alpha_init = 0.1
        best_stage_val = 1e300
        best_stage_x = x.copy()
        no_improve = 0

        for t in range(n_iters_per_beta):
            g = lse_grad_nb(y, P, beta)
            x_new, fval_new, alpha_used = armijo_step_nb(y, g, P, beta, alpha_init)
            alpha_init = min(alpha_used * 2.0, 1.0)

            # Nesterov momentum
            momentum = t / (t + 3.0)
            n = len(x_new)
            y_new = np.empty(n)
            for i in range(n):
                y_new[i] = x_new[i] + momentum * (x_new[i] - x_prev[i])
            y_new = project_simplex_nb(y_new)

            x_prev = x_new.copy()
            x = x_new
            y = y_new

            true_val = np.max(autoconv_coeffs(x, P))
            if true_val < best_stage_val:
                best_stage_val = true_val
                best_stage_x = x.copy()
                no_improve = 0
            else:
                no_improve += 1

            if no_improve > 1000:
                break

        x = best_stage_x

    true_val = np.max(autoconv_coeffs(x, P))
    if true_val < best_true_val:
        best_true_val = true_val
        best_x = x.copy()

    return best_true_val, best_x


# Warm up the single-restart function
_bs = np.array([1.0, 10.0, 100.0])
_ = _lse_single_restart(np.ones(5) / 5.0, 5, _bs, 10)


def optimize_lse_parallel(P, beta_schedule, n_iters_per_beta=10000,
                           n_restarts=30, n_jobs=-1, verbose=True):
    """Parallel LSE Nesterov: each restart runs on a separate CPU core."""
    beta_arr = np.array(beta_schedule, dtype=np.float64)

    # Pre-generate all random initializations (Dirichlet)
    rng = np.random.default_rng()
    inits = [rng.dirichlet(np.ones(P)) for _ in range(n_restarts)]

    # Run restarts in parallel across cores
    results = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(_lse_single_restart)(inits[i], P, beta_arr, n_iters_per_beta)
        for i in range(n_restarts)
    )

    # Find best across all restarts
    best_val = np.inf
    best_x = None
    for i, (val, x) in enumerate(results):
        if verbose and (i % 10 == 0 or val < best_val):
            tag = "  <-- best" if (best_x is None or val < best_val) else ""
            print(f"    Restart {i:>3}: {val:.6f}{tag}")
        if val < best_val:
            best_val = val
            best_x = x.copy()

    return best_val, best_x


print("Parallel LSE Nesterov optimizer defined.")

## Polyak Subgradient Optimizer

Polyak step-size subgradient method for the non-smooth $\max_k c_k$ objective.
Uses a target value to compute step sizes. Wrapped in a parallel multi-restart,
multi-target driver.

In [ ]:
@nb.njit(cache=True)
def _polyak_single_restart(x_init, P, target, n_iters):
    """Run one Polyak subgradient trajectory. Returns (best_val, best_x)."""
    x = x_init.copy()
    n = len(x)
    best_val = 1e300
    best_x = x.copy()

    for t in range(n_iters):
        c = autoconv_coeffs(x, P)
        fval = np.max(c)
        if fval < best_val:
            best_val = fval
            best_x = x.copy()

        # Find argmax
        k_star = 0
        for k in range(len(c)):
            if c[k] > c[k_star]:
                k_star = k

        # Subgradient
        g = np.zeros(n)
        for i in range(n):
            j = k_star - i
            if 0 <= j < n:
                g[i] = 2.0 * (2.0 * P) * x[j]

        gnorm2 = 0.0
        for i in range(n):
            gnorm2 += g[i] * g[i]
        if gnorm2 < 1e-20:
            break

        step = (fval - target) / gnorm2
        if step < 0.0:
            step = 1e-4 / (1.0 + t)

        for i in range(n):
            x[i] = x[i] - step * g[i]
        x = project_simplex_nb(x)

    return best_val, best_x


# Warm up
_ = _polyak_single_restart(np.ones(5) / 5.0, 5, 1.5, 10)


def polyak_parallel(P, n_iters=100000, n_restarts=30,
                    targets=(1.51, 1.505, 1.50, 1.495, 1.49, 1.48),
                    n_jobs=-1, verbose=True):
    """Parallel Polyak subgradient across all (target, restart) pairs."""
    rng = np.random.default_rng()

    # Build all (init, target) pairs
    tasks = []
    for target in targets:
        for _ in range(n_restarts):
            tasks.append((rng.dirichlet(np.ones(P)), target))

    results = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(_polyak_single_restart)(init, P, target, n_iters)
        for init, target in tasks
    )

    best_val = np.inf
    best_x = None
    for val, x in results:
        if val < best_val:
            best_val = val
            best_x = x.copy()

    if verbose:
        print(f"    Best across {len(tasks)} runs: {best_val:.6f}")

    return best_val, best_x


print("Parallel Polyak optimizer defined.")

## LSE vs Polyak: Head-to-Head Comparison

Run both optimizers across multiple P values (10 to 200) with 30 restarts each,
then compare exact peak autoconvolution values.

In [ ]:
P_values = [10, 20, 30, 50, 75, 100, 150, 200]
beta_schedule = [1, 2, 4, 8, 15, 30, 60, 100, 150, 250, 400, 600, 1000, 1500, 2000]

results = {}

for P in P_values:
    print(f"\n{'='*60}")
    print(f"P = {P}")
    print(f"{'='*60}")

    edges = np.linspace(-0.25, 0.25, P + 1)
    bin_width = 0.5 / P

    # LSE Nesterov (30 restarts, parallel)
    t0 = time.time()
    print(f"  LSE Nesterov (30 restarts, 15 stages, 10k iters/stage):")
    val_lse, x_lse = optimize_lse_parallel(
        P, beta_schedule, n_iters_per_beta=10000, n_restarts=30
    )
    exact_lse = exact_val(x_lse, P)
    dt_lse = time.time() - t0
    print(f"  => LSE exact = {exact_lse:.6f}  ({dt_lse:.1f}s)")

    # Polyak (180 runs, parallel)
    t0 = time.time()
    print(f"  Polyak (180 runs, 100k iters):")
    val_poly, x_poly = polyak_parallel(P, n_iters=100000, n_restarts=30)
    exact_poly = exact_val(x_poly, P)
    dt_poly = time.time() - t0
    print(f"  => Polyak exact = {exact_poly:.6f}  ({dt_poly:.1f}s)")

    delta = exact_lse - exact_poly
    winner = "LSE" if delta < 0 else "Polyak" if delta > 0 else "Tie"
    print(f"  >> Winner: {winner} (delta = {delta:+.6f})")

    results[P] = {
        'lse': (exact_lse, x_lse),
        'polyak': (exact_poly, x_poly),
    }

In [ ]:
# === Results table ===

print(f"{'P':>5} | {'LSE (exact)':>14} | {'Polyak (exact)':>14} | {'Delta':>10} | {'Winner':>8}")
print('-' * 62)
for P in P_values:
    r = results[P]
    v_lse = r['lse'][0]
    v_poly = r['polyak'][0]
    delta = v_lse - v_poly
    winner = "LSE" if delta < 0 else "Polyak" if delta > 0 else "Tie"
    print(f"{P:>5} | {v_lse:>14.6f} | {v_poly:>14.6f} | {delta:>+10.6f} | {winner:>8}")

print(f"\nBest known upper bound: 1.5029")

## Visualization: LSE vs Polyak

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for idx, P in enumerate(P_values):
    ax = axes[idx]
    edges = np.linspace(-0.25, 0.25, P + 1)
    centers = 0.5 * (edges[:-1] + edges[1:])
    bin_width = 0.5 / P

    x_lse = results[P]['lse'][1]
    x_poly = results[P]['polyak'][1]

    ax.bar(centers - bin_width*0.15, x_lse / bin_width, width=bin_width*0.35,
           alpha=0.7, label='LSE', color='C0', edgecolor='k', linewidth=0.3)
    ax.bar(centers + bin_width*0.15, x_poly / bin_width, width=bin_width*0.35,
           alpha=0.7, label='Polyak', color='C1', edgecolor='k', linewidth=0.3)

    v_lse = results[P]['lse'][0]
    v_poly = results[P]['polyak'][0]
    ax.set_title(f'P={P}\nLSE={v_lse:.4f}, Poly={v_poly:.4f}', fontsize=9)
    if idx == 0:
        ax.legend(fontsize=7)

plt.tight_layout()
plt.suptitle('LSE vs Polyak: step function heights', y=1.02, fontsize=14)
plt.savefig('logsumexp_vs_polyak_heights.png', dpi=150, bbox_inches='tight')
plt.show()

# Convergence comparison
fig, ax = plt.subplots(figsize=(8, 5))
Ps = sorted(results.keys())
lse_vals = [results[P]['lse'][0] for P in Ps]
poly_vals = [results[P]['polyak'][0] for P in Ps]

ax.plot(Ps, lse_vals, 'o-', label='LSE continuation', markersize=8)
ax.plot(Ps, poly_vals, 's--', label='Polyak subgradient', markersize=8)
ax.axhline(1.5029, color='r', linestyle='--', alpha=0.5, label='Best known (1.5029)')
ax.set_xlabel('P (number of bins)')
ax.set_ylabel('Peak autoconvolution (exact)')
ax.set_title('LSE continuation vs Polyak subgradient')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('logsumexp_vs_polyak_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

## Ablation: Beta Schedule

Test different $\beta$-continuation schedules at P=50 to understand the sensitivity
of LSE continuation to the schedule choice.

In [ ]:
P_abl = 50
schedules = {
    'aggressive': np.array([1., 10., 100., 1000.]),
    'moderate': np.array([1., 2., 5., 10., 20., 50., 100., 200., 500., 1000.]),
    'fine_15': np.array([1., 2., 4., 8., 15., 30., 60., 100., 150., 250., 400., 600., 1000., 1500., 2000.]),
    'gentle': np.array([1., 1.5, 2., 3., 5., 7., 10., 15., 20., 30., 50., 75., 100., 150., 200., 300., 500., 750., 1000.]),
    'fixed_low': np.full(10, 10.0),
    'fixed_high': np.full(10, 1000.0),
}

print(f"Beta schedule ablation at P = {P_abl} (20 restarts each, parallel)")
print(f"{'Schedule':>15} | {'Stages':>6} | {'Exact obj':>12}")
print('-' * 42)

for name, sched in schedules.items():
    val, x = optimize_lse_parallel(
        P_abl, sched, n_iters_per_beta=8000, n_restarts=20, verbose=False
    )
    exact = exact_val(x, P_abl)
    print(f"{name:>15} | {len(sched):>6} | {exact:>12.6f}")

## Hybrid Optimizer: LSE Continuation + Adaptive Polyak Polish

Two-phase approach:
1. **Phase 1 (LSE):** Find a good basin via smooth $\beta$-continuation with Nesterov acceleration
2. **Phase 2 (Polyak):** Polish with adaptive Polyak subgradient using a shrinking target offset

In [ ]:
@nb.njit(cache=True)
def _polyak_polish_nb(x_init, P, n_iters):
    """Adaptive Polyak polish: target = best_val - shrinking_offset."""
    x = x_init.copy()
    n = len(x)
    best_val = np.max(autoconv_coeffs(x, P))
    best_x = x.copy()

    for t in range(n_iters):
        c = autoconv_coeffs(x, P)
        fval = np.max(c)
        if fval < best_val:
            best_val = fval
            best_x = x.copy()

        offset = 0.01 / (1.0 + t * 1e-4)
        target = best_val - offset

        k_star = 0
        for k in range(len(c)):
            if c[k] > c[k_star]:
                k_star = k

        g = np.zeros(n)
        for i in range(n):
            j = k_star - i
            if 0 <= j < n:
                g[i] = 2.0 * (2.0 * P) * x[j]

        gnorm2 = 0.0
        for i in range(n):
            gnorm2 += g[i] * g[i]
        if gnorm2 < 1e-20:
            break

        step = (fval - target) / gnorm2
        if step < 0.0:
            step = 1e-5 / (1.0 + t * 1e-4)

        for i in range(n):
            x[i] = x[i] - step * g[i]
        x = project_simplex_nb(x)

    return best_val, best_x


@nb.njit(cache=True)
def _hybrid_single_restart(x_init, P, beta_schedule, n_iters_lse, n_iters_polyak):
    """One restart: LSE Nesterov continuation -> adaptive Polyak polish."""
    x = x_init.copy()

    # Phase 1: LSE continuation
    for stage in range(len(beta_schedule)):
        beta = beta_schedule[stage]
        y = x.copy()
        x_prev = x.copy()
        alpha_init = 0.1
        best_stage_val = 1e300
        best_stage_x = x.copy()
        no_improve = 0

        for t in range(n_iters_lse):
            g = lse_grad_nb(y, P, beta)
            x_new, fval_new, alpha_used = armijo_step_nb(y, g, P, beta, alpha_init)
            alpha_init = min(alpha_used * 2.0, 1.0)

            momentum = t / (t + 3.0)
            n = len(x_new)
            y_new = np.empty(n)
            for i in range(n):
                y_new[i] = x_new[i] + momentum * (x_new[i] - x_prev[i])
            y_new = project_simplex_nb(y_new)

            x_prev = x_new.copy()
            x = x_new
            y = y_new

            tv = np.max(autoconv_coeffs(x, P))
            if tv < best_stage_val:
                best_stage_val = tv
                best_stage_x = x.copy()
                no_improve = 0
            else:
                no_improve += 1
            if no_improve > 800:
                break

        x = best_stage_x

    lse_val = np.max(autoconv_coeffs(x, P))

    # Phase 2: adaptive Polyak polish
    polished_val, polished_x = _polyak_polish_nb(x, P, n_iters_polyak)

    return lse_val, polished_val, polished_x


# Warm up
_bs = np.array([1.0, 10.0])
_ = _polyak_polish_nb(np.ones(5) / 5.0, 5, 10)
_ = _hybrid_single_restart(np.ones(5) / 5.0, 5, _bs, 10, 10)


def hybrid_parallel(P, beta_schedule, n_iters_lse=10000,
                     n_iters_polyak=100000, n_restarts=30, n_jobs=-1, verbose=True):
    """Parallel hybrid: LSE basin-finding + adaptive Polyak polishing."""
    beta_arr = np.array(beta_schedule, dtype=np.float64)
    rng = np.random.default_rng()
    inits = [rng.dirichlet(np.ones(P)) for _ in range(n_restarts)]

    results = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(_hybrid_single_restart)(inits[i], P, beta_arr, n_iters_lse, n_iters_polyak)
        for i in range(n_restarts)
    )

    best_val = np.inf
    best_x = None
    for i, (lse_v, pol_v, x) in enumerate(results):
        if verbose and (i % 10 == 0 or pol_v < best_val):
            tag = "  <-- best" if (best_x is None or pol_v < best_val) else ""
            print(f"    Restart {i:>3}: LSE={lse_v:.6f} -> polished={pol_v:.6f}{tag}")
        if pol_v < best_val:
            best_val = pol_v
            best_x = x.copy()

    return best_val, best_x


print("Parallel hybrid optimizer defined.")

## Three-Method Head-to-Head: LSE vs Polyak vs Hybrid

Run all three methods at P = 20, 50, 100, 200 and compare.

In [ ]:
P_values_hybrid = [20, 50, 100, 200]
beta_sched = [1, 2, 4, 8, 15, 30, 60, 100, 150, 250, 400, 600, 1000, 1500, 2000]

results_hybrid = {}

for P in P_values_hybrid:
    print(f"\n{'='*60}")
    print(f"P = {P}")
    print(f"{'='*60}")
    edges = np.linspace(-0.25, 0.25, P + 1)
    bin_width = 0.5 / P

    # Hybrid (30 restarts, LSE -> adaptive Polyak, parallel)
    t0 = time.time()
    print(f"  Hybrid (30 restarts, parallel):")
    val_hyb, x_hyb = hybrid_parallel(
        P, beta_sched, n_iters_lse=10000, n_iters_polyak=100000, n_restarts=30
    )
    exact_hyb = exact_val(x_hyb, P)
    dt = time.time() - t0
    print(f"  => Hybrid exact = {exact_hyb:.6f}  ({dt:.1f}s)")

    exact_lse = results.get(P, {}).get('lse', (None,))[0]
    exact_poly = results.get(P, {}).get('polyak', (None,))[0]

    results_hybrid[P] = {
        'hybrid': exact_hyb,
        'lse_only': exact_lse,
        'polyak_only': exact_poly,
    }

# === Final comparison table ===
print(f"\n\n{'='*72}")
print(f"FINAL RESULTS")
print(f"{'='*72}")
print(f"{'P':>5} | {'Hybrid':>12} | {'LSE only':>12} | {'Polyak only':>12} | {'Best known':>12}")
print('-' * 68)
for P in P_values_hybrid:
    r = results_hybrid[P]
    hyb_s = f"{r['hybrid']:>12.6f}" if r['hybrid'] is not None else f"{'N/A':>12}"
    lse_s = f"{r['lse_only']:>12.6f}" if r['lse_only'] is not None else f"{'N/A':>12}"
    pol_s = f"{r['polyak_only']:>12.6f}" if r['polyak_only'] is not None else f"{'N/A':>12}"
    print(f"{P:>5} | {hyb_s} | {lse_s} | {pol_s} | {1.5029:>12.4f}")

## Save Initial Best Solutions

In [ ]:
# Collect best solution per P across all methods
all_best = {}

# From LSE-only and Polyak-only
for P in P_values:
    edges = np.linspace(-0.25, 0.25, P + 1)
    bin_width = 0.5 / P
    for method_name, key in [('lse', 'lse'), ('polyak', 'polyak')]:
        exact_v, x_simplex = results[P][key]
        entry = {
            'method': method_name,
            'P': P,
            'claimed_exact_peak': exact_v,
            'simplex_weights': x_simplex.tolist(),
            'edges': edges.tolist(),
            'heights': (x_simplex / bin_width).tolist(),
        }
        label = f"{method_name}_P{P}"
        all_best[label] = entry

# From hybrid -- re-run to get x vectors (they were not stored in results_hybrid)
print("Re-running hybrid at key P values to save solutions...")
beta_sched_save = [1, 2, 4, 8, 15, 30, 60, 100, 150, 250, 400, 600, 1000, 1500, 2000]
for P in P_values_hybrid:
    edges = np.linspace(-0.25, 0.25, P + 1)
    bin_width = 0.5 / P
    val_hyb, x_hyb = hybrid_parallel(
        P, beta_sched_save, n_iters_lse=10000, n_iters_polyak=100000,
        n_restarts=30, verbose=False
    )
    exact_hyb = exact_val(x_hyb, P)
    entry = {
        'method': 'hybrid',
        'P': P,
        'claimed_exact_peak': exact_hyb,
        'simplex_weights': x_hyb.tolist(),
        'edges': edges.tolist(),
        'heights': (x_hyb / bin_width).tolist(),
    }
    all_best[f"hybrid_P{P}"] = entry
    print(f"  hybrid P={P}: {exact_hyb:.6f}")

# Find the overall best solution
best_label = min(all_best, key=lambda k: all_best[k]['claimed_exact_peak'])
best_entry = all_best[best_label]
print(f"\nOverall best: {best_label} -> {best_entry['claimed_exact_peak']:.6f}")

# Save everything
out_path = os.path.join('.', 'best_solutions.json')
with open(out_path, 'w') as f:
    json.dump(all_best, f, indent=2)
print(f"Saved {len(all_best)} solutions to {out_path}")

# Heavy Compute: Multi-Strategy Hybrid Search (~2-3 hours)

We test **12 different initialization strategies** for the hybrid (LSE continuation
-> Polyak polish) optimizer, across multiple P values, with many restarts and
extended iterations.

## Initialization Strategies
1. **dirichlet_uniform** -- Dirichlet(alpha=1) (current baseline)
2. **dirichlet_sparse** -- Dirichlet(alpha=0.1), favors sparse solutions
3. **dirichlet_concentrated** -- Dirichlet(alpha=5), near-uniform
4. **gaussian_peak** -- Gaussian centered at 0, random width
5. **bimodal** -- Two Gaussians at +/-offset
6. **cosine_shaped** -- cos(pi*x/0.25) profile
7. **triangle** -- Triangle centered at 0
8. **flat_noisy** -- Uniform + small perturbation
9. **boundary_heavy** -- Extra weight near +/-0.25 edges
10. **random_sparse_k** -- Only k random bins nonzero
11. **symmetric_dirichlet** -- Enforce f(x)=f(-x) symmetry
12. **warm_perturb** -- Best known solution + Gaussian noise

## Initialization Strategy Generators and Extended Hybrid Runner

In [ ]:
def make_inits(strategy, P, n_restarts, rng, warm_x=None):
    """Generate n_restarts initial simplex vectors for a given strategy."""
    centers = np.linspace(-0.25 + 0.25/P, 0.25 - 0.25/P, P)
    inits = []

    for _ in range(n_restarts):
        if strategy == 'dirichlet_uniform':
            x = rng.dirichlet(np.ones(P))

        elif strategy == 'dirichlet_sparse':
            x = rng.dirichlet(np.full(P, 0.1))

        elif strategy == 'dirichlet_concentrated':
            x = rng.dirichlet(np.full(P, 5.0))

        elif strategy == 'gaussian_peak':
            sigma = rng.uniform(0.03, 0.15)
            mu = rng.uniform(-0.05, 0.05)
            x = np.exp(-0.5 * ((centers - mu) / sigma) ** 2)
            x += rng.uniform(0, 0.01, P)  # small noise floor
            x /= x.sum()

        elif strategy == 'bimodal':
            sep = rng.uniform(0.05, 0.2)
            sigma = rng.uniform(0.02, 0.08)
            ratio = rng.uniform(0.3, 0.7)
            x = ratio * np.exp(-0.5 * ((centers - sep/2) / sigma) ** 2)
            x += (1 - ratio) * np.exp(-0.5 * ((centers + sep/2) / sigma) ** 2)
            x += rng.uniform(0, 0.005, P)
            x /= x.sum()

        elif strategy == 'cosine_shaped':
            phase = rng.uniform(0, np.pi)
            freq = rng.uniform(0.5, 3.0)
            x = np.cos(freq * np.pi * centers / 0.25 + phase) ** 2
            x += rng.uniform(0, 0.02, P)
            x /= x.sum()

        elif strategy == 'triangle':
            peak_pos = rng.uniform(-0.1, 0.1)
            width = rng.uniform(0.1, 0.25)
            x = np.maximum(0, 1 - np.abs(centers - peak_pos) / width)
            x += rng.uniform(0, 0.01, P)
            x /= x.sum()

        elif strategy == 'flat_noisy':
            noise_scale = rng.uniform(0.01, 0.2)
            x = np.ones(P) + noise_scale * rng.standard_normal(P)
            x = np.maximum(x, 0.0)
            x /= x.sum()

        elif strategy == 'boundary_heavy':
            # Extra mass near edges of [-0.25, 0.25]
            decay = rng.uniform(2.0, 10.0)
            x = np.exp(-decay * np.abs(np.abs(centers) - 0.25))
            x += rng.uniform(0, 0.01, P)
            x /= x.sum()

        elif strategy == 'random_sparse_k':
            k = rng.integers(max(3, P // 10), max(4, P // 3))
            x = np.zeros(P)
            idx = rng.choice(P, size=k, replace=False)
            x[idx] = rng.dirichlet(np.ones(k))

        elif strategy == 'symmetric_dirichlet':
            half = P // 2
            x_half = rng.dirichlet(np.ones(half))
            x = np.zeros(P)
            x[:half] = x_half
            x[P - half:] = x_half[::-1]
            if P % 2 == 1:
                x[half] = rng.uniform(0.0, 0.1)
            x /= x.sum()

        elif strategy == 'warm_perturb':
            if warm_x is not None:
                noise = rng.uniform(0.3, 1.5)
                x = warm_x.copy() + noise * rng.standard_normal(P) * np.mean(warm_x)
                x = np.maximum(x, 0.0)
                if x.sum() < 1e-12:
                    x = rng.dirichlet(np.ones(P))
                else:
                    x /= x.sum()
            else:
                x = rng.dirichlet(np.ones(P))

        else:
            raise ValueError(f"Unknown strategy: {strategy}")

        inits.append(x)

    return inits


def hybrid_strategy_run(P, strategy, beta_schedule, n_iters_lse=15000,
                        n_iters_polyak=200000, n_restarts=80,
                        n_jobs=-1, warm_x=None, verbose=True):
    """Run hybrid optimizer with a specific initialization strategy."""
    beta_arr = np.array(beta_schedule, dtype=np.float64)
    rng = np.random.default_rng()
    inits = make_inits(strategy, P, n_restarts, rng, warm_x=warm_x)

    results = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(_hybrid_single_restart)(inits[i], P, beta_arr, n_iters_lse, n_iters_polyak)
        for i in range(n_restarts)
    )

    best_val = np.inf
    best_x = None
    all_vals = []
    for i, (lse_v, pol_v, x) in enumerate(results):
        all_vals.append(pol_v)
        if pol_v < best_val:
            best_val = pol_v
            best_x = x.copy()

    if verbose:
        arr = np.array(all_vals)
        print(f"    {strategy:30s}  best={best_val:.6f}  "
              f"median={np.median(arr):.6f}  std={np.std(arr):.6f}")

    return best_val, best_x, all_vals


def upsample_solution(x_low, P_low, P_high):
    """Upsample a P_low-bin solution to P_high bins via linear interpolation."""
    edges_low = np.linspace(-0.25, 0.25, P_low + 1)
    edges_high = np.linspace(-0.25, 0.25, P_high + 1)
    bin_width_low = 0.5 / P_low
    bin_width_high = 0.5 / P_high

    # Convert to density, interpolate, convert back to simplex weights
    heights_low = x_low / bin_width_low
    centers_low = 0.5 * (edges_low[:-1] + edges_low[1:])
    centers_high = 0.5 * (edges_high[:-1] + edges_high[1:])

    heights_high = np.interp(centers_high, centers_low, heights_low)
    heights_high = np.maximum(heights_high, 0.0)

    x_high = heights_high * bin_width_high
    if x_high.sum() > 0:
        x_high /= x_high.sum()
    else:
        x_high = np.ones(P_high) / P_high
    return x_high


# Quick validation of initialization strategies
rng_test = np.random.default_rng(0)
strategies = [
    'dirichlet_uniform', 'dirichlet_sparse', 'dirichlet_concentrated',
    'gaussian_peak', 'bimodal', 'cosine_shaped', 'triangle',
    'flat_noisy', 'boundary_heavy', 'random_sparse_k',
    'symmetric_dirichlet', 'warm_perturb',
]
for s in strategies:
    inits = make_inits(s, 50, 3, rng_test, warm_x=np.ones(50)/50)
    for x in inits:
        assert np.allclose(x.sum(), 1.0, atol=1e-10), f"{s}: sum={x.sum()}"
        assert np.all(x >= -1e-15), f"{s}: has negatives"

print(f"All {len(strategies)} initialization strategies verified.")
print("Extended hybrid runner and multi-scale upsampler defined.")

## Round 1: Strategy Sweep at P=200

Test all 12 initialization strategies with 80 restarts each to identify the
most promising strategies for deeper compute.

In [ ]:
# Extended beta schedule with finer stages
beta_heavy = [1, 1.5, 2, 3, 5, 8, 12, 18, 28, 42, 65, 100, 150, 230, 350,
              500, 750, 1000, 1500, 2000, 3000]

strategies_all = [
    'dirichlet_uniform', 'dirichlet_sparse', 'dirichlet_concentrated',
    'gaussian_peak', 'bimodal', 'cosine_shaped', 'triangle',
    'flat_noisy', 'boundary_heavy', 'random_sparse_k',
    'symmetric_dirichlet', 'warm_perturb',
]

# Global tracker
global_best = {'val': np.inf, 'x': None, 'P': None, 'strategy': None, 'round': None}
round_results = {}

def update_global_best(val, x, P, strategy, round_name):
    if val < global_best['val']:
        global_best['val'] = val
        global_best['x'] = x.copy()
        global_best['P'] = P
        global_best['strategy'] = strategy
        global_best['round'] = round_name
        print(f"  *** NEW GLOBAL BEST: {val:.6f} (P={P}, {strategy}, {round_name}) ***")

t_total_start = time.time()

# --- Round 1: P=200, all strategies, 80 restarts each ---
P = 200
print(f"\n{'='*70}")
print(f"ROUND 1: Strategy sweep at P={P}  (80 restarts x 12 strategies)")
print(f"{'='*70}")
t0 = time.time()

r1_results = {}
for strat in strategies_all:
    val, x, vals = hybrid_strategy_run(
        P, strat, beta_heavy,
        n_iters_lse=15000, n_iters_polyak=200000,
        n_restarts=80, warm_x=global_best.get('x')
    )
    ev = exact_val(x, P)
    r1_results[strat] = {'val': ev, 'x': x, 'all_vals': vals}
    update_global_best(ev, x, P, strat, 'round1_P200')

# Rank strategies
ranked = sorted(r1_results.items(), key=lambda kv: kv[1]['val'])
print(f"\nRound 1 ranking (P={P}):")
for i, (s, r) in enumerate(ranked):
    print(f"  {i+1:>2}. {s:30s}  {r['val']:.6f}")
top_strategies = [s for s, _ in ranked[:6]]  # top 6

dt = time.time() - t0
print(f"\nRound 1 completed in {dt:.0f}s ({dt/60:.1f} min)")
round_results['round1'] = {s: r1_results[s]['val'] for s in strategies_all}

## Round 2: Top Strategies at P=300

In [ ]:
P = 300
print(f"\n{'='*70}")
print(f"ROUND 2: Top strategies at P={P}  (100 restarts x {len(top_strategies)} strategies)")
print(f"{'='*70}")
t0 = time.time()

# Upsample best known to P=300 for warm starts
warm_300 = upsample_solution(global_best['x'], global_best['P'], P) if global_best['x'] is not None else None

r2_results = {}
for strat in top_strategies:
    val, x, vals = hybrid_strategy_run(
        P, strat, beta_heavy,
        n_iters_lse=15000, n_iters_polyak=300000,
        n_restarts=100, warm_x=warm_300
    )
    ev = exact_val(x, P)
    r2_results[strat] = {'val': ev, 'x': x}
    update_global_best(ev, x, P, strat, 'round2_P300')

# Also run warm_perturb with the current best
val, x, vals = hybrid_strategy_run(
    P, 'warm_perturb', beta_heavy,
    n_iters_lse=15000, n_iters_polyak=300000,
    n_restarts=100, warm_x=warm_300
)
ev = exact_val(x, P)
r2_results['warm_perturb'] = {'val': ev, 'x': x}
update_global_best(ev, x, P, 'warm_perturb', 'round2_P300')

dt = time.time() - t0
print(f"\nRound 2 completed in {dt:.0f}s ({dt/60:.1f} min)")
print(f"Global best so far: {global_best['val']:.6f} (P={global_best['P']}, {global_best['strategy']})")

# Update top strategies based on round 2
ranked_r2 = sorted(r2_results.items(), key=lambda kv: kv[1]['val'])
top_strategies_r2 = [s for s, _ in ranked_r2[:4]]
print(f"Top strategies after round 2: {top_strategies_r2}")

## Round 3: Top Strategies at P=500

In [ ]:
P = 500
print(f"\n{'='*70}")
print(f"ROUND 3: Top strategies at P={P}  (80 restarts x {len(top_strategies_r2)} strategies + warm_perturb)")
print(f"{'='*70}")
t0 = time.time()

warm_500 = upsample_solution(global_best['x'], global_best['P'], P) if global_best['x'] is not None else None

r3_results = {}
strats_r3 = list(set(top_strategies_r2 + ['warm_perturb', 'dirichlet_uniform']))
for strat in strats_r3:
    val, x, vals = hybrid_strategy_run(
        P, strat, beta_heavy,
        n_iters_lse=15000, n_iters_polyak=300000,
        n_restarts=80, warm_x=warm_500
    )
    ev = exact_val(x, P)
    r3_results[strat] = {'val': ev, 'x': x}
    update_global_best(ev, x, P, strat, 'round3_P500')

dt = time.time() - t0
print(f"\nRound 3 completed in {dt:.0f}s ({dt/60:.1f} min)")
print(f"Global best so far: {global_best['val']:.6f} (P={global_best['P']}, {global_best['strategy']})")

## Round 4: Multi-Scale Warm-Start Cascade (200 -> 300 -> 500 -> 750)

Take the best solutions from earlier rounds, upsample them to higher P values,
and polish heavily.

In [ ]:
print(f"\n{'='*70}")
print(f"ROUND 4: Multi-scale warm-start cascade (200->300->500->750)")
print(f"{'='*70}")
t0 = time.time()

# Collect best solutions at each P from previous rounds
best_per_P = {}
for strat, r in r1_results.items():
    P_s = 200
    if P_s not in best_per_P or r['val'] < best_per_P[P_s][0]:
        best_per_P[P_s] = (r['val'], r['x'])

for strat, r in r2_results.items():
    P_s = 300
    if P_s not in best_per_P or r['val'] < best_per_P[P_s][0]:
        best_per_P[P_s] = (r['val'], r['x'])

for strat, r in r3_results.items():
    P_s = 500
    if P_s not in best_per_P or r['val'] < best_per_P[P_s][0]:
        best_per_P[P_s] = (r['val'], r['x'])

# Cascade: at each level, take the best from below, upsample, then run
# warm_perturb + dirichlet_uniform on it
cascade_Ps = [300, 500, 750]

for P_target in cascade_Ps:
    # Find best solution at the nearest lower P
    source_Ps = sorted([p for p in best_per_P if p < P_target], reverse=True)
    if not source_Ps:
        continue

    P_source = source_Ps[0]
    _, x_source = best_per_P[P_source]
    warm_x = upsample_solution(x_source, P_source, P_target)

    print(f"\n  Cascade: P={P_source} -> P={P_target}")

    # Heavy warm_perturb run
    val, x, _ = hybrid_strategy_run(
        P_target, 'warm_perturb', beta_heavy,
        n_iters_lse=20000, n_iters_polyak=500000,
        n_restarts=120, warm_x=warm_x
    )
    ev = exact_val(x, P_target)
    update_global_best(ev, x, P_target, 'warm_perturb', f'round4_cascade_P{P_target}')
    if P_target not in best_per_P or ev < best_per_P[P_target][0]:
        best_per_P[P_target] = (ev, x)

    # Also some fresh random starts
    val, x, _ = hybrid_strategy_run(
        P_target, 'dirichlet_uniform', beta_heavy,
        n_iters_lse=20000, n_iters_polyak=500000,
        n_restarts=60
    )
    ev = exact_val(x, P_target)
    update_global_best(ev, x, P_target, 'dirichlet_uniform', f'round4_cascade_P{P_target}')
    if ev < best_per_P[P_target][0]:
        best_per_P[P_target] = (ev, x)

dt = time.time() - t0
print(f"\nRound 4 completed in {dt:.0f}s ({dt/60:.1f} min)")
print(f"Global best so far: {global_best['val']:.6f} (P={global_best['P']}, {global_best['strategy']})")

## Round 5: Maximum Compute at P=500 and P=750

In [ ]:
print(f"\n{'='*70}")
print(f"ROUND 5: Maximum compute at P=500 and P=750")
print(f"{'='*70}")
t0 = time.time()

# Even longer beta schedule
beta_ultra = [1, 1.3, 1.7, 2.2, 3, 4, 5.5, 7.5, 10, 14, 20, 28, 40, 55,
              75, 100, 140, 200, 280, 400, 560, 800, 1100, 1500, 2000, 3000, 4000]

for P in [500, 750]:
    print(f"\n  --- P={P} ---")

    # Warm start from best available
    if P in best_per_P:
        warm_x = best_per_P[P][1]
    elif global_best['x'] is not None:
        warm_x = upsample_solution(global_best['x'], global_best['P'], P)
    else:
        warm_x = None

    # Heavy warm_perturb: 150 restarts, max iterations
    val, x, _ = hybrid_strategy_run(
        P, 'warm_perturb', beta_ultra,
        n_iters_lse=20000, n_iters_polyak=500000,
        n_restarts=150, warm_x=warm_x
    )
    ev = exact_val(x, P)
    update_global_best(ev, x, P, 'warm_perturb', f'round5_P{P}')
    if P not in best_per_P or ev < best_per_P[P][0]:
        best_per_P[P] = (ev, x)

    # Heavy dirichlet_uniform: 100 restarts
    val, x, _ = hybrid_strategy_run(
        P, 'dirichlet_uniform', beta_ultra,
        n_iters_lse=20000, n_iters_polyak=500000,
        n_restarts=100
    )
    ev = exact_val(x, P)
    update_global_best(ev, x, P, 'dirichlet_uniform', f'round5_P{P}')
    if ev < best_per_P[P][0]:
        best_per_P[P] = (ev, x)

    # Heavy gaussian_peak: 80 restarts
    val, x, _ = hybrid_strategy_run(
        P, 'gaussian_peak', beta_ultra,
        n_iters_lse=20000, n_iters_polyak=500000,
        n_restarts=80
    )
    ev = exact_val(x, P)
    update_global_best(ev, x, P, 'gaussian_peak', f'round5_P{P}')
    if ev < best_per_P[P][0]:
        best_per_P[P] = (ev, x)

    # Heavy symmetric: 80 restarts
    val, x, _ = hybrid_strategy_run(
        P, 'symmetric_dirichlet', beta_ultra,
        n_iters_lse=20000, n_iters_polyak=500000,
        n_restarts=80
    )
    ev = exact_val(x, P)
    update_global_best(ev, x, P, 'symmetric_dirichlet', f'round5_P{P}')
    if ev < best_per_P[P][0]:
        best_per_P[P] = (ev, x)

dt = time.time() - t0
print(f"\nRound 5 completed in {dt:.0f}s ({dt/60:.1f} min)")
print(f"Global best so far: {global_best['val']:.6f} (P={global_best['P']}, {global_best['strategy']})")

## Round 6: Final Push at P=1000

In [ ]:
P = 1000
print(f"\n{'='*70}")
print(f"ROUND 6: Final push at P={P}")
print(f"{'='*70}")
t0 = time.time()

# Upsample best from highest available P
warm_1000 = upsample_solution(best_per_P[max(best_per_P.keys())][1],
                               max(best_per_P.keys()), P)

# Heavy warm_perturb
val, x, _ = hybrid_strategy_run(
    P, 'warm_perturb', beta_ultra,
    n_iters_lse=15000, n_iters_polyak=500000,
    n_restarts=100, warm_x=warm_1000
)
ev = exact_val(x, P)
update_global_best(ev, x, P, 'warm_perturb', 'round6_P1000')
if P not in best_per_P or ev < best_per_P[P][0]:
    best_per_P[P] = (ev, x)

# Fresh random starts
val, x, _ = hybrid_strategy_run(
    P, 'dirichlet_uniform', beta_ultra,
    n_iters_lse=15000, n_iters_polyak=500000,
    n_restarts=60
)
ev = exact_val(x, P)
update_global_best(ev, x, P, 'dirichlet_uniform', 'round6_P1000')
if ev < best_per_P[P][0]:
    best_per_P[P] = (ev, x)

# Gaussian peaks
val, x, _ = hybrid_strategy_run(
    P, 'gaussian_peak', beta_ultra,
    n_iters_lse=15000, n_iters_polyak=500000,
    n_restarts=60
)
ev = exact_val(x, P)
update_global_best(ev, x, P, 'gaussian_peak', 'round6_P1000')
if ev < best_per_P[P][0]:
    best_per_P[P] = (ev, x)

dt = time.time() - t0
print(f"\nRound 6 completed in {dt:.0f}s ({dt/60:.1f} min)")
print(f"Global best so far: {global_best['val']:.6f} (P={global_best['P']}, {global_best['strategy']})")

## Round 7: Cross-Pollination

Blend the best solutions from different P values and strategies via random convex
combinations, then re-optimize. This explores the space between known good solutions.

In [ ]:
print(f"\n{'='*70}")
print(f"ROUND 7: Cross-pollination (blend best solutions + re-optimize)")
print(f"{'='*70}")
t0 = time.time()

# For each target P, create blended initializations from multiple source P levels
for P_target in [500, 750, 1000]:
    print(f"\n  --- Cross-pollination at P={P_target} ---")

    # Collect all available solutions, upsample to P_target
    upsampled = []
    for P_src, (v, x_src) in best_per_P.items():
        if P_src != P_target:
            x_up = upsample_solution(x_src, P_src, P_target)
        else:
            x_up = x_src.copy()
        upsampled.append(x_up)

    if len(upsampled) < 2:
        continue

    # Generate blended inits: random convex combinations of 2-3 solutions
    rng = np.random.default_rng()
    n_blend = 100
    blend_inits = []
    for _ in range(n_blend):
        k = rng.integers(2, min(4, len(upsampled) + 1))
        idxs = rng.choice(len(upsampled), size=k, replace=False)
        weights = rng.dirichlet(np.ones(k))
        x_blend = sum(w * upsampled[i] for i, w in zip(idxs, weights))
        # Add small noise
        x_blend += 0.3 * rng.standard_normal(P_target) * np.mean(x_blend)
        x_blend = np.maximum(x_blend, 0.0)
        x_blend /= x_blend.sum()
        blend_inits.append(x_blend)

    # Run hybrid on blended inits
    beta_arr = np.array(beta_ultra, dtype=np.float64)
    results_blend = Parallel(n_jobs=-1, verbose=0)(
        delayed(_hybrid_single_restart)(blend_inits[i], P_target, beta_arr, 15000, 500000)
        for i in range(n_blend)
    )

    best_val = np.inf
    best_x = None
    for lse_v, pol_v, x in results_blend:
        if pol_v < best_val:
            best_val = pol_v
            best_x = x.copy()

    ev = exact_val(best_x, P_target)
    print(f"    Blended best: {ev:.6f}")
    update_global_best(ev, best_x, P_target, 'cross_pollination', f'round7_P{P_target}')
    if P_target not in best_per_P or ev < best_per_P[P_target][0]:
        best_per_P[P_target] = (ev, best_x)

dt = time.time() - t0
print(f"\nRound 7 completed in {dt:.0f}s ({dt/60:.1f} min)")
print(f"Global best so far: {global_best['val']:.6f} (P={global_best['P']}, {global_best['strategy']})")

## Final Summary and Save

In [ ]:
t_total = time.time() - t_total_start
print(f"\n{'='*70}")
print(f"HEAVY COMPUTE COMPLETE -- Total time: {t_total:.0f}s ({t_total/60:.1f} min, {t_total/3600:.2f} hr)")
print(f"{'='*70}")

# Summary table
print(f"\n{'P':>6} | {'Best value':>14} | {'Source':>20}")
print('-' * 50)
for P_s in sorted(best_per_P.keys()):
    v, _ = best_per_P[P_s]
    print(f"{P_s:>6} | {v:>14.6f} |")

print(f"\n{'='*70}")
print(f"GLOBAL BEST: {global_best['val']:.6f}")
print(f"  P = {global_best['P']}")
print(f"  Strategy = {global_best['strategy']}")
print(f"  Round = {global_best['round']}")
print(f"  Best known in literature: 1.5029")
print(f"  Gap to literature: {global_best['val'] - 1.5029:+.6f}")
print(f"{'='*70}")

# Save all best solutions
save_data = {}
for P_s in sorted(best_per_P.keys()):
    v, x = best_per_P[P_s]
    edges = np.linspace(-0.25, 0.25, P_s + 1)
    bin_width = 0.5 / P_s
    save_data[f"heavy_P{P_s}"] = {
        'P': P_s,
        'exact_peak': v,
        'simplex_weights': x.tolist(),
        'edges': edges.tolist(),
        'heights': (x / bin_width).tolist(),
    }

# Also save global best separately
save_data['global_best'] = {
    'P': global_best['P'],
    'exact_peak': global_best['val'],
    'strategy': global_best['strategy'],
    'round': global_best['round'],
    'simplex_weights': global_best['x'].tolist(),
}

out_path = os.path.join('.', 'best_solutions.json')
with open(out_path, 'w') as f:
    json.dump(save_data, f, indent=2)
print(f"\nSaved {len(save_data)} solutions to {out_path}")

# Visualization of global best
P_best = global_best['P']
x_best = global_best['x']
edges = np.linspace(-0.25, 0.25, P_best + 1)
centers = 0.5 * (edges[:-1] + edges[1:])
bin_width = 0.5 / P_best

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(centers, x_best / bin_width, width=bin_width * 0.9,
        color='C0', edgecolor='k', linewidth=0.2, alpha=0.8)
ax1.set_title(f'Global best: P={P_best}, peak={global_best["val"]:.6f}\n'
              f'Strategy: {global_best["strategy"]}, Round: {global_best["round"]}')
ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')

# Convergence across P values
Ps_sorted = sorted(best_per_P.keys())
vals_sorted = [best_per_P[p][0] for p in Ps_sorted]
ax2.plot(Ps_sorted, vals_sorted, 'o-', markersize=8, linewidth=2)
ax2.axhline(1.5029, color='r', linestyle='--', alpha=0.5, label='Literature best (1.5029)')
ax2.set_xlabel('P (number of bins)')
ax2.set_ylabel('Peak autoconvolution (exact)')
ax2.set_title('Best value vs discretization')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('heavy_compute_results.png', dpi=150, bbox_inches='tight')
plt.show()